In [1]:
from toy_classify import *
import ot
import os
import warnings
warnings.filterwarnings('ignore')

## CoDSA

In [2]:
list_result = []
list_origin = []
W_list = [[] for j in range(10)]
ce_nf=[]



num_sim=10
num_split= len(ratio_list )
best_indices = np.empty((num_sim, num_split, 2), dtype=int)
# best_test_errors[i,j] will store the corresponding test error.
best_test_errors = np.empty((num_sim, num_split))



# Set data sizes.
n_minority = 800
n_majority = 3200
n_val = 400
n_test = 800
n_train =4000

#hyperparameter sets
ratio_list = [x/10 for x in list(range(1,11))] #split ratio r
candidate_m_syn=  [int(n_train*round(x * 0.1, 2)) for x in range(21)] #synthetic sample size m/n
candidate_alpha_scale= [0.1, 0.2, 0.3,0.4, 0.5,0.6, 0.7,0.8, 0.9] #alpha




X_total, y_total, regions_total = generate_imbalanced_data(
    n_minority + n_val + n_test, 
    n_majority + n_val + n_test, 
    seed=0
)
y_total = y_total.reshape(-1, 1)


# Create directories for saving models and synthetic data.

if not os.path.exists("synthetic_nf"):
    os.makedirs("synthetic_nf")

for j in range(10):
    set_seed(j)
    
    # Separate minority and majority indices based on region labels.
    minority_idx = np.where(regions_total == 0)[0]
    majority_idx = np.where(regions_total == 1)[0]

    # Shuffle indices separately.
    np.random.shuffle(minority_idx)
    np.random.shuffle(majority_idx)

    # For minority:
    min_train_idx = minority_idx[:n_minority]
    min_val_idx = minority_idx[n_minority : n_minority + n_val]
    min_test_idx = minority_idx[n_minority + n_val : n_minority + n_val + n_test]

    # For majority:
    maj_train_idx = majority_idx[:n_majority]
    maj_val_idx = majority_idx[n_majority : n_majority + n_val]
    maj_test_idx = majority_idx[n_majority + n_val : n_majority + n_val + n_test]

    # Combine indices for training, validation, test.
    X_train_orig = np.vstack((X_total[min_train_idx], X_total[maj_train_idx]))
    y_train_orig = np.vstack((y_total[min_train_idx], y_total[maj_train_idx]))
    regions_train = np.concatenate((regions_total[min_train_idx], regions_total[maj_train_idx]))

    X_val = np.vstack((X_total[min_val_idx], X_total[maj_val_idx]))
    y_val = np.vstack((y_total[min_val_idx], y_total[maj_val_idx]))
    regions_val = np.concatenate((regions_total[min_val_idx], regions_total[maj_val_idx]))

    X_test = np.vstack((X_total[min_test_idx], X_total[maj_test_idx]))
    y_test = np.vstack((y_total[min_test_idx], y_total[maj_test_idx]))
    regions_test = np.concatenate((regions_total[min_test_idx], regions_total[maj_test_idx]))

    # Combine training X and y for diffusion training.
    XY_train_orig = combine_XY(X_train_orig, y_train_orig)
    n_train = XY_train_orig.shape[0]

    origin_model, mse_val_origin, mse_test_origin, y_val_opred, y_test_opred = train_and_evaluate(
        X_train_orig, y_train_orig, X_val, y_val, X_test, y_test,
        input_dim=X_train_orig.shape[1], batch_size=1024, patience=20
    )
    
    list_origin.append([mse_val_origin, mse_test_origin])
    
    all_result = []
    for split_ratio in ratio_list:
        n_diff1 = int(split_ratio * n_minority)
        n_diff2 = int(split_ratio * n_majority)

        # Shuffle indices for each group.
        indices_min = np.arange(n_minority)
        np.random.shuffle(indices_min)
        indices_maj = np.arange(n_majority)
        np.random.shuffle(indices_maj)

        # Extract diffusion subset using the shuffled indices.
        XY_diff = np.vstack((
            XY_train_orig[indices_min[:n_diff1], :],
            XY_train_orig[n_minority + indices_maj[:n_diff2], :]
        ))
        regions_diff = np.concatenate((
            regions_train[indices_min[:n_diff1]],
            regions_train[n_minority + indices_maj[:n_diff2]]
        ))
        
        if split_ratio == 1:
            XY_reg = None
        else:
            XY_reg = np.vstack((XY_train_orig[n_diff1:n_minority, :],
                                XY_train_orig[(n_minority+n_diff2):, :]))
            
        candidate_m_syn=  [int(n_train*round(x * 0.1, 2)) for x in range(21)]
        #[int(n_train*round(x * 0.05, 2)) for x in range(21)]+[int(n_train*x) for x in [1.5,2]]#[int(n_train*k) for k in [0.1, 0.2, 0.3,0.4, 0.5,0.6, 0.7,0.8, 0.9, 1.0]]     
        candidate_alpha_scale= [0.1, 0.2, 0.3,0.4, 0.5,0.6, 0.7,0.8, 0.9]
        m=2*max(candidate_m_syn)
            
        result=[]
        
        file_path = f"synthetic_nf/synthetic_X_seed{j}_ratio{split_ratio:.1f}.npy"
        if os.path.exists(file_path):          

            # Build file paths.
            file_X = f"synthetic_nf/synthetic_X_seed{j}_ratio{split_ratio:.1f}.npy"
            file_y = f"synthetic_nf/synthetic_y_seed{j}_ratio{split_ratio:.1f}.npy"

            # Load the synthetic data.
            X_syn_full = np.load(file_X)
            y_syn_full = np.load(file_y)
        else:
            
            U_diff = torch.tensor(XY_diff[:, :-1], dtype=torch.float32).to('cuda')

            # Initialize and train the conditional diffusion model.
            model = ConditionalDiffusionModel(
                text_dim=X_total.shape[1], cond_dim=1, hidden_dim=1024, time_embed_dim=128,
                num_fc_blocks=10, dropout=0.00001
            )
            sampler = DDIMSampler(device='cuda', noise_steps=1000)
            model = model.to('cuda')
            trained_model = train_conditional_diffusion(
                model, U_diff, regions_diff, sampler, num_epochs=3000, batch_size=128, seed=j, device='cuda'
            )
            
            # Generate synthetic data using the trained diffusion model.
            X_syn_full, y_syn_full = generate(trained_model, None, m, 0.5, sampler, seed=j)

            # Save synthetic data.
            np.save(f"synthetic_nf/synthetic_X_seed{j}_ratio{split_ratio:.1f}.npy", X_syn_full)
            np.save(f"synthetic_nf/synthetic_y_seed{j}_ratio{split_ratio:.1f}.npy", y_syn_full)

        X_truth, y_truth, _ = generate_imbalanced_data(int(m/2), int(m/2), seed=j)
        truth_samples = combine_XY(X_truth, y_truth.reshape(-1, 1))
        gen_samples = combine_XY(X_syn_full, y_syn_full)
        
        # Compute cost matrix (Euclidean distances) and Wasserstein distance.
        M = ot.dist(truth_samples, gen_samples, metric='euclidean')
        a = np.ones((m,)) / m
        b = np.ones((m,)) / m
        W_distance = ot.emd2(a, b, M)
        W_list[j].append(W_distance)
        print("Wasserstein distance:", W_distance)
        
        for m_syn in candidate_m_syn:
            tmp = []
            if m_syn == 0 and split_ratio == 1:
                for alpha_scale in candidate_alpha_scale:
                    tmp.append([np.inf, np.inf])
                result.append(tmp)
                continue
            for alpha_scale in candidate_alpha_scale:
                if m_syn == 0:
                    X_train_combined = XY_reg[:, :-1]
                    y_train_combined = XY_reg[:, -1:]
                else:
                    X_train_combined = np.vstack((
                        X_syn_full[:int(m_syn*alpha_scale), :],
                        X_syn_full[int(m/2):int(m/2+m_syn*(1-alpha_scale)), :]
                    ))
                    y_train_combined = np.vstack((
                        y_syn_full[:int(m_syn*alpha_scale), :],
                        y_syn_full[int(m/2):int(m/2+m_syn*(1-alpha_scale)), :]
                    ))
                    if split_ratio < 1:
                        X_train_combined = np.vstack((X_train_combined, XY_reg[:, :-1]))
                        y_train_combined = np.vstack((y_train_combined, XY_reg[:, -1:]))
                
                _, mse_val, mse_test, _, _ = train_and_evaluate(
                    X_train_combined, y_train_combined, X_val, y_val, X_test, y_test,
                    input_dim=X_train_orig.shape[1], batch_size=1024, patience=20
                )
                print(f"m_syn={m_syn}, alpha_scale={alpha_scale} -> Validation MSE: {mse_val:.4f} -> Test MSE: {mse_test:.4f}")
                tmp.append([mse_val, mse_test])
            result.append(tmp)
        all_result.append(result)
        
    ax= [all_result[x][:21] for x in range(10)] 
    ax=np.array(ax)
    num_sim, num_split, num_m, num_min_ratio = ax.shape
    val_errors = ax[: , :, :, 0]

    # Find the indices (k, l) that minimize the validation error.
    o, k, l = np.unravel_index(np.argmin(val_errors), val_errors.shape)


    best_ratio = ratio_list[o]
    best_m = candidate_m_syn[k]
    best_alpha = candidate_alpha_scale[l]

    
    n_diff1 = int(best_ratio * n_minority)
    n_diff2 = int(best_ratio * n_majority)

    # Shuffle indices for each group.
    indices_min = np.arange(n_minority)
    np.random.shuffle(indices_min)
    indices_maj = np.arange(n_majority)
    np.random.shuffle(indices_maj)

    # Extract diffusion subset using the shuffled indices.
    XY_diff = np.vstack((
        XY_train_orig[indices_min[:n_diff1], :],
        XY_train_orig[n_minority + indices_maj[:n_diff2], :]
    ))
    regions_diff = np.concatenate((
        regions_train[indices_min[:n_diff1]],
        regions_train[n_minority + indices_maj[:n_diff2]]
    ))

    XY_reg = np.vstack((XY_train_orig[n_diff1:n_minority,:],XY_train_orig[(n_minority+n_diff2):,:]))


    m=2*max(candidate_m_syn)

    # Build file paths.
    file_X = f"synthetic_nf/synthetic_X_seed{j}_ratio{best_ratio:.1f}.npy"
    file_y = f"synthetic_nf/synthetic_y_seed{j}_ratio{best_ratio:.1f}.npy"

    # Load the synthetic data.
    X_syn_full = np.load(file_X)
    y_syn_full = np.load(file_y)

    X_train_combined = np.vstack((
        X_syn_full[:int(best_m*best_alpha), :],
        X_syn_full[int(m/2):int(m/2+best_m*(1-best_alpha)), :]
    ))
    y_train_combined = np.vstack((
        y_syn_full[:int(best_m*best_alpha), :],
        y_syn_full[int(m/2):int(m/2+best_m*(1-best_alpha)), :]
    ))

    X_train_combined = np.vstack((X_train_combined, XY_reg[:, :-1]))
    y_train_combined = np.vstack((y_train_combined, XY_reg[:, -1:]))

    ce_one={}
    # Train and evaluate on validation set
    _, mse_val, mse_test, ypred_val, ypred_test = train_and_evaluate(
        X_train_combined, y_train_combined, X_val, y_val, X_test, y_test,
        input_dim= X_train_orig.shape[1], batch_size =1024, patience =20)
    ce_one['avg']=mse_test
    
    _, mse_val, mse_test, ypred_val, ypred_test = train_and_evaluate(
    X_train_combined, y_train_combined, X_val, y_val, X_test[regions_test==1,], y_test[regions_test==1,],
    input_dim= X_train_orig.shape[1], batch_size =1024, patience =20)
    ce_one['major']=mse_test
        
        
    _, mse_val, mse_test, ypred_val, ypred_test = train_and_evaluate(
    X_train_combined, y_train_combined, X_val, y_val, X_test[regions_test==0,], y_test[regions_test==0,],
    input_dim= X_train_orig.shape[1], batch_size =1024, patience =20)
    ce_one['minor']=mse_test       
    
    ce_nf.append(ce_one)
        
    list_result.append(all_result)

Early stopping at epoch 67
Validation Accuracy: 0.71
Test Accuracy: 0.729375
Validation BCE Metric: 0.49700456857681274
Test BCE Metric: 0.5028505325317383
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=6800, alpha_scale=0.8 -> Validation MSE: 0.4970 -> Test MSE: 0.5029
Early stopping at epoch 58
Validation Accuracy: 0.69625
Test Accuracy: 0.7
Validation BCE Metric: 0.5602867603302002
Test BCE Metric: 0.5704314112663269
Average Validation Entropy: 0.3892269730567932
Average Test Entropy: nan
m_syn=6800, alpha_scale=0.9 -> Validation MSE: 0.5603 -> Test MSE: 0.5704
Early stopping at epoch 87
Validation Accuracy: 0.67375
Test Accuracy: 0.674375
Validation BCE Metric: 0.6638872623443604
Test BCE Metric: 0.696656346321106
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=7200, alpha_scale=0.1 -> Validation MSE: 0.6639 -> Test MSE: 0.6967
Early stopping at epoch 87
Validation Accuracy: 0.70625
Test Accuracy: 0.714375
Validation BCE Metric: 0.5374417901039124
T

/users/1/tianx/.local/lib/python3.8/site-packages/ot/lp/__init__.py:630: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  check_result(result_code)


Wasserstein distance: 6.573571709198452
Early stopping at epoch 21
Validation Accuracy: 0.50125
Test Accuracy: 0.495625
Validation BCE Metric: 0.7336056232452393
Test BCE Metric: 0.7674868702888489
Average Validation Entropy: 0.6500977873802185
Average Test Entropy: nan
m_syn=0, alpha_scale=0.1 -> Validation MSE: 0.7336 -> Test MSE: 0.7675
Early stopping at epoch 21
Validation Accuracy: 0.50125
Test Accuracy: 0.495625
Validation BCE Metric: 0.7336056232452393
Test BCE Metric: 0.7674868702888489
Average Validation Entropy: 0.6500977873802185
Average Test Entropy: nan
m_syn=0, alpha_scale=0.2 -> Validation MSE: 0.7336 -> Test MSE: 0.7675
Early stopping at epoch 21
Validation Accuracy: 0.50125
Test Accuracy: 0.495625
Validation BCE Metric: 0.7336056232452393
Test BCE Metric: 0.7674868702888489
Average Validation Entropy: 0.6500977873802185
Average Test Entropy: nan
m_syn=0, alpha_scale=0.3 -> Validation MSE: 0.7336 -> Test MSE: 0.7675
Early stopping at epoch 21
Validation Accuracy: 0.5012

Early stopping at epoch 28
Validation Accuracy: 0.50625
Test Accuracy: 0.505
Validation BCE Metric: 0.7602014541625977
Test BCE Metric: 0.7806269526481628
Average Validation Entropy: 0.5884389281272888
Average Test Entropy: 0.5835584998130798
m_syn=1200, alpha_scale=0.2 -> Validation MSE: 0.7602 -> Test MSE: 0.7806
Validation Accuracy: 0.68625
Test Accuracy: 0.69
Validation BCE Metric: 0.5676517486572266
Test BCE Metric: 0.6152342557907104
Average Validation Entropy: 0.4351567029953003
Average Test Entropy: nan
m_syn=1200, alpha_scale=0.3 -> Validation MSE: 0.5677 -> Test MSE: 0.6152
Early stopping at epoch 87
Validation Accuracy: 0.70875
Test Accuracy: 0.69375
Validation BCE Metric: 0.5256373286247253
Test BCE Metric: 0.5649595856666565
Average Validation Entropy: 0.4706023335456848
Average Test Entropy: nan
m_syn=1200, alpha_scale=0.4 -> Validation MSE: 0.5256 -> Test MSE: 0.5650
Early stopping at epoch 92
Validation Accuracy: 0.7375
Test Accuracy: 0.728125
Validation BCE Metric: 0.4

Validation Accuracy: 0.73375
Test Accuracy: 0.711875
Validation BCE Metric: 0.49021080136299133
Test BCE Metric: 0.5503461956977844
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=2400, alpha_scale=0.4 -> Validation MSE: 0.4902 -> Test MSE: 0.5503
Validation Accuracy: 0.75
Test Accuracy: 0.7425
Validation BCE Metric: 0.4756962060928345
Test BCE Metric: 0.5177047848701477
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=2400, alpha_scale=0.5 -> Validation MSE: 0.4757 -> Test MSE: 0.5177
Validation Accuracy: 0.74375
Test Accuracy: 0.733125
Validation BCE Metric: 0.4731186628341675
Test BCE Metric: 0.5348148941993713
Average Validation Entropy: nan
Average Test Entropy: nan
m_syn=2400, alpha_scale=0.6 -> Validation MSE: 0.4731 -> Test MSE: 0.5348
Early stopping at epoch 86
Validation Accuracy: 0.70375
Test Accuracy: 0.714375
Validation BCE Metric: 0.49055227637290955
Test BCE Metric: 0.5347866415977478
Average Validation Entropy: nan
Average Test Entropy: na

...

Epoch 290/3000, Loss: 0.2131
Epoch 291/3000, Loss: 0.2200
Epoch 292/3000, Loss: 0.2249
Epoch 293/3000, Loss: 0.2169
Epoch 294/3000, Loss: 0.2166
Epoch 295/3000, Loss: 0.2231
Epoch 296/3000, Loss: 0.2242
Epoch 297/3000, Loss: 0.2292
Epoch 298/3000, Loss: 0.2191
Epoch 299/3000, Loss: 0.2206
Epoch 300/3000, Loss: 0.2223
Epoch 301/3000, Loss: 0.2235
Epoch 302/3000, Loss: 0.2287
Epoch 303/3000, Loss: 0.2144
Epoch 304/3000, Loss: 0.2248
Epoch 305/3000, Loss: 0.2311
Epoch 306/3000, Loss: 0.2212
Epoch 307/3000, Loss: 0.2156
Epoch 308/3000, Loss: 0.2198
Epoch 309/3000, Loss: 0.2204
Epoch 310/3000, Loss: 0.2251
Epoch 311/3000, Loss: 0.2124
Epoch 312/3000, Loss: 0.2217
Epoch 313/3000, Loss: 0.2187
Epoch 314/3000, Loss: 0.2204
Epoch 315/3000, Loss: 0.2200
Epoch 316/3000, Loss: 0.2158
Epoch 317/3000, Loss: 0.2142
Epoch 318/3000, Loss: 0.2341
Epoch 319/3000, Loss: 0.2154
Epoch 320/3000, Loss: 0.2193
Epoch 321/3000, Loss: 0.2168
Epoch 322/3000, Loss: 0.2237
Epoch 323/3000, Loss: 0.2156
Epoch 324/3000

Epoch 575/3000, Loss: 0.2016
Epoch 576/3000, Loss: 0.2181
Epoch 577/3000, Loss: 0.2053
Epoch 578/3000, Loss: 0.1989
Epoch 579/3000, Loss: 0.2077
Epoch 580/3000, Loss: 0.2151
Epoch 581/3000, Loss: 0.2108
Epoch 582/3000, Loss: 0.2067
Epoch 583/3000, Loss: 0.2030
Epoch 584/3000, Loss: 0.2124
Epoch 585/3000, Loss: 0.2011
Epoch 586/3000, Loss: 0.2003
Epoch 587/3000, Loss: 0.2093
Epoch 588/3000, Loss: 0.2015
Epoch 589/3000, Loss: 0.1985
Epoch 590/3000, Loss: 0.2121
Epoch 591/3000, Loss: 0.2193
Epoch 592/3000, Loss: 0.2124
Epoch 593/3000, Loss: 0.2213
Epoch 594/3000, Loss: 0.2112
Epoch 595/3000, Loss: 0.2159
Epoch 596/3000, Loss: 0.2075
Epoch 597/3000, Loss: 0.2061
Epoch 598/3000, Loss: 0.2080
Epoch 599/3000, Loss: 0.2043
Epoch 600/3000, Loss: 0.2132
Epoch 601/3000, Loss: 0.2129
Epoch 602/3000, Loss: 0.2117
Epoch 603/3000, Loss: 0.2016
Epoch 604/3000, Loss: 0.2105
Epoch 605/3000, Loss: 0.2092
Epoch 606/3000, Loss: 0.2082
Epoch 607/3000, Loss: 0.2055
Epoch 608/3000, Loss: 0.2041
Epoch 609/3000

Epoch 859/3000, Loss: 0.2014
Epoch 860/3000, Loss: 0.2032
Epoch 861/3000, Loss: 0.2088
Epoch 862/3000, Loss: 0.1992
Epoch 863/3000, Loss: 0.2086
Epoch 864/3000, Loss: 0.1984
Epoch 865/3000, Loss: 0.1932
Epoch 866/3000, Loss: 0.2011
Epoch 867/3000, Loss: 0.2035
Epoch 868/3000, Loss: 0.2119
Epoch 869/3000, Loss: 0.2016
Epoch 870/3000, Loss: 0.1949
Epoch 871/3000, Loss: 0.1934
Epoch 872/3000, Loss: 0.1993
Epoch 873/3000, Loss: 0.2068
Epoch 874/3000, Loss: 0.2002
Epoch 875/3000, Loss: 0.2014
Epoch 876/3000, Loss: 0.1968
Epoch 877/3000, Loss: 0.2122
Epoch 878/3000, Loss: 0.2099
Epoch 879/3000, Loss: 0.2020
Epoch 880/3000, Loss: 0.1987
Epoch 881/3000, Loss: 0.1972
Epoch 882/3000, Loss: 0.1990
Epoch 883/3000, Loss: 0.1930
Epoch 884/3000, Loss: 0.2001
Epoch 885/3000, Loss: 0.1996
Epoch 886/3000, Loss: 0.1952
Epoch 887/3000, Loss: 0.1973
Epoch 888/3000, Loss: 0.2038
Epoch 889/3000, Loss: 0.1972
Epoch 890/3000, Loss: 0.2063
Epoch 891/3000, Loss: 0.2003
Epoch 892/3000, Loss: 0.2031
Epoch 893/3000

Epoch 1138/3000, Loss: 0.1954
Epoch 1139/3000, Loss: 0.2075
Epoch 1140/3000, Loss: 0.1928
Epoch 1141/3000, Loss: 0.1890
Epoch 1142/3000, Loss: 0.2023
Epoch 1143/3000, Loss: 0.1956
Epoch 1144/3000, Loss: 0.1957
Epoch 1145/3000, Loss: 0.1893
Epoch 1146/3000, Loss: 0.1961
Epoch 1147/3000, Loss: 0.2024
Epoch 1148/3000, Loss: 0.1987
Epoch 1149/3000, Loss: 0.1935
Epoch 1150/3000, Loss: 0.2076


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
import pickle

with open("result/res_nf.pkl", "wb") as f:
    pickle.dump([list_result,W_list,list_origin,ce_nf], f)

### Summary statistics for Table 1

In [3]:
[np.mean([x['major'] for x in ce_nf]),np.mean([x['minor'] for x in ce_nf]),np.mean([x['avg'] for x in ce_nf])] 

[0.4441003352403641, 0.48407823741436007, 0.46408929526805875]

In [4]:
[np.std([x['major'] for x in ce_nf]),np.std([x['minor'] for x in ce_nf]),np.std([x['avg'] for x in ce_nf])] 

[0.06251783587619346, 0.05232663369809709, 0.016654522137394885]